# Day 12: Passage Pathing
https://adventofcode.com/2021/day/12

### Part 1
Find all paths that only go through small caves, i.e., lower case node names, once.

In [1]:
import os
import networkx as nx 
from collections import Counter

In [2]:
# input data.
def test_input_location(
    file_loc: str = 'test_input.txt', 
    data_directory: str  = 'data/day_12'
) -> str:
    return os.path.join(data_directory, file_loc)

def test_input_location_slightly(
    file_loc: str = 'test_slightly_longer.txt', 
    data_directory: str  = 'data/day_12'
) -> str:
    return os.path.join(data_directory, file_loc)

def test_input_location_even(
    file_loc: str = 'test_even_longer.txt', 
    data_directory: str  = 'data/day_12'
) -> str:
    return os.path.join(data_directory, file_loc)

def input_location(
    file_loc: str = 'input.txt', 
    data_directory: str  = 'data/day_12'
) -> str:
    return os.path.join(data_directory, file_loc)

def create_graph(input_file:str):
    g = nx.Graph()
    if os.path.exists(input_file):
        with open(input_file) as f:    
            for row, line in enumerate(f):
                if line.rstrip():
                    caves = sorted(line.strip().split('-'))
                    g.add_edge(caves[0], caves[1])
    return g


In [3]:
def drunken_walk(g:nx.Graph, node:str, prior_nodes: list[str]): 
    """
    Recursive walk through the network (<nx.Graph>) that allows us to visit
    nodes with lower case only once, and the rest infinite times.
    Returns,
        List of List[str] for all possible paths.
    """
    possible_steps = [
        n 
        for n in g.neighbors(node) 
        if n.isupper() or n not in prior_nodes
    ]

    new_prior_nodes = prior_nodes + [node]
    ending_chains:list[list[str]] = []

    for step in possible_steps:
        if step == 'end':
            ending_chains.append([node, step])
        else:
            possible_chains = drunken_walk(g, step, new_prior_nodes)
            for chain in possible_chains:
                ending_chains.append([node] + chain)
    
    return ending_chains

In [4]:
g = create_graph(test_input_location())
paths = drunken_walk(g, 'start', [])
assert len(paths) == 10
sorted(paths)

[['start', 'A', 'b', 'A', 'c', 'A', 'end'],
 ['start', 'A', 'b', 'A', 'end'],
 ['start', 'A', 'b', 'end'],
 ['start', 'A', 'c', 'A', 'b', 'A', 'end'],
 ['start', 'A', 'c', 'A', 'b', 'end'],
 ['start', 'A', 'c', 'A', 'end'],
 ['start', 'A', 'end'],
 ['start', 'b', 'A', 'c', 'A', 'end'],
 ['start', 'b', 'A', 'end'],
 ['start', 'b', 'end']]

In [5]:
g = create_graph(input_location())
paths = drunken_walk(g, 'start', [])
len(paths)

3761

### Part 2

In [6]:
def visited_a_small_cave_2x_already(c: Counter) -> bool:
    """
    Given a counter, we determine if we have already visited a small cave more than once.
    """
    return bool(len([k for k,v in c.items() if k.islower() and v > 1]))

def drunken_walk_part2(g:nx.Graph, node:str, prior_nodes: Counter): 
    """
    Recursive walk through the network (<nx.Graph>) that allows us to visit
    1 lower case 2x (except start) the rest of lower case nodes only once, and the 
    rest infinite times.
    Returns,
        List of List[str] for all possible paths.
    """

    # Have we visited a small cave already?
    small_cave_2x =  visited_a_small_cave_2x_already(prior_nodes + Counter([node]))

    # What are the possible next steps we can take.
    possible_steps = [
        n 
        for n in g.neighbors(node)
        if (
            n != 'start'
            and (
                n.isupper() 
                or not small_cave_2x
                or (small_cave_2x and n not in prior_nodes.elements())
            )
        )
    ]

    new_prior_nodes: Counter = prior_nodes + Counter([node])
    ending_chains:list[list[str]] = []

    for step in possible_steps:
        if step == 'end':
            ending_chains.append([node, step])
        else:
            possible_chains = drunken_walk_part2(g, step, new_prior_nodes)
            for chain in possible_chains:
                ending_chains.append([node] + chain)
    
    return ending_chains

In [7]:
assert visited_a_small_cave_2x_already(Counter(['start', 'A', 'c', 'c', 'A', 'b', 'end'])) == True
assert visited_a_small_cave_2x_already(Counter(['start', 'A', 'c', 'A', 'b', 'end'])) == False

In [8]:
g = create_graph(test_input_location())
c = Counter()
paths = drunken_walk_part2(g, 'start', c)
assert len(paths) == 36
sorted(paths)

[['start', 'A', 'b', 'A', 'b', 'A', 'c', 'A', 'end'],
 ['start', 'A', 'b', 'A', 'b', 'A', 'end'],
 ['start', 'A', 'b', 'A', 'b', 'end'],
 ['start', 'A', 'b', 'A', 'c', 'A', 'b', 'A', 'end'],
 ['start', 'A', 'b', 'A', 'c', 'A', 'b', 'end'],
 ['start', 'A', 'b', 'A', 'c', 'A', 'c', 'A', 'end'],
 ['start', 'A', 'b', 'A', 'c', 'A', 'end'],
 ['start', 'A', 'b', 'A', 'end'],
 ['start', 'A', 'b', 'd', 'b', 'A', 'c', 'A', 'end'],
 ['start', 'A', 'b', 'd', 'b', 'A', 'end'],
 ['start', 'A', 'b', 'd', 'b', 'end'],
 ['start', 'A', 'b', 'end'],
 ['start', 'A', 'c', 'A', 'b', 'A', 'b', 'A', 'end'],
 ['start', 'A', 'c', 'A', 'b', 'A', 'b', 'end'],
 ['start', 'A', 'c', 'A', 'b', 'A', 'c', 'A', 'end'],
 ['start', 'A', 'c', 'A', 'b', 'A', 'end'],
 ['start', 'A', 'c', 'A', 'b', 'd', 'b', 'A', 'end'],
 ['start', 'A', 'c', 'A', 'b', 'd', 'b', 'end'],
 ['start', 'A', 'c', 'A', 'b', 'end'],
 ['start', 'A', 'c', 'A', 'c', 'A', 'b', 'A', 'end'],
 ['start', 'A', 'c', 'A', 'c', 'A', 'b', 'end'],
 ['start', 'A', 

In [9]:
g = create_graph(test_input_location_slightly())
paths = drunken_walk_part2(g, 'start', Counter())
assert len(paths) == 103
# sorted(paths)


In [10]:
g = create_graph(test_input_location_even())
paths = drunken_walk_part2(g, 'start', Counter())
assert len(paths)  == 3509

In [11]:
g = create_graph(input_location())
paths = drunken_walk_part2(g, 'start', Counter())
len(paths)

99138